In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

lam0=1
s0=0

def stages(sfinal):
    """define sfinal as a function of q0"""
    def ss(q0):
        for c in range(5):
            def lam(c):
                if c==0:
                    return lam0
                return lam(c-1)/2
            def s(c):
                if c==0:
                    return s0
                return s(c-1)-math.log(1-(lam(c)/q(c-1)))/lam(c)
            def q(c):
                if c==0:
                    return q0
                return math.exp(-lam(c)*(s(c)-s(c-1)))*q(c-1)
        return s(5)

    """find the corresponding q0 for a given sfinal"""
    from scipy.optimize import newton
    def sss(i):
        return ss(i)-sfinal
    trueq0=newton(sss, 4)

    """calculate s0-s5 with the derived q0"""
    for c in range(5):
        def lam(c):
            if c==0:
                return lam0
            return lam(c-1)/2
        def sstar(c):
            if c==0:
                return s0
            return sstar(c-1)-math.log(1-(lam(c)/qstar(c-1)))/lam(c)
        def qstar(c):
            if c==0:
                return trueq0
            return math.exp(-lam(c)*(sstar(c)-sstar(c-1)))*qstar(c-1)
    S=[]
    Q=[]
    for c in range(6):
        S.append(sstar(c))
        Q.append(qstar(c))
    return S

"""draw plots for s1-s5; w1-w5"""
fig, ax=plt.subplots()
ssfinal=np.linspace(0.1,1,100)
for i in [1,2,3,4,5]:
    sst=[]
    for sfinal in ssfinal:
        sst.append(stages(sfinal)[i])
    def ssst(i):
        return sst
    current_label=r's{}'.format(i)
    ax.plot(ssfinal, sst, linewidth=2, alpha=0.6, label=current_label)
    plt.fill_between(ssfinal,0, ssst(i), color='red', alpha=.25)
ax.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

lam0=1
s0=0
p0=0

def wages(sfinal):
    """first solve equations (3)&(4)"""
    def stages(sfinal):

        """define sfinal as a function of q0"""
        def ss(q0):
            for c in range(5):
                def lam(c):
                    if c==0:
                        return lam0
                    return lam(c-1)/2
                def s(c):
                    if c==0:
                        return s0
                    return s(c-1)-math.log(1-(lam(c)/q(c-1)))/lam(c)
                def q(c):
                    if c==0:
                        return q0
                    return math.exp(-lam(c)*(s(c)-s(c-1)))*q(c-1)
            return s(5)

        """find the corresponding q0 for a given sfinal"""
        from scipy.optimize import newton
        def sss(i):
            return ss(i)-sfinal
        trueq0=newton(sss, 4)

        """calculate s0-s5 with the derived q0"""
        for c in range(5):
            def lam(c):
                if c==0:
                    return lam0
                return lam(c-1)/2
            def sstar(c):
                if c==0:
                    return s0
                return sstar(c-1)-math.log(1-(lam(c)/qstar(c-1)))/lam(c)
            def qstar(c):
                if c==0:
                    return trueq0
                return math.exp(-lam(c)*(sstar(c)-sstar(c-1)))*qstar(c-1)
        S=[]
        Q=[]
        for c in range(6):
            S.append(sstar(c))
            Q.append(qstar(c))
        return S

    '''derive the production measure of every country'''
    def N(sfinal):
        n=[0]
        for i in [1,2,3,4,5]:
            n.append(stages(sfinal)[i]-stages(sfinal)[i-1])
        return n
    
    """then solve (5)&(6), a coupled linear difference equations"""
    def A(i):
        def lam(c):
            if c==0:
                return lam0
            return lam(c-1)/2
        """generate coefficient matrix"""
        a11=1+(lam(i)-lam(i+1))*(math.exp(lam(i)*N(sfinal)[i])-1)/lam(i)
        a12=(lam(i)-lam(i+1))*math.exp(lam(i)*N(sfinal)[i])
        a21=math.exp(lam(i)*N(sfinal)[i])
        a22=(math.exp(lam(i)*N(sfinal)[i])-1)/lam(i)
        a=((a11,a12),(a21,a22))
        return np.array(a)
    def pfinal(v0):
        def x(i):
            if i==0:
                return np.array((v0,0))
            return np.dot(A(i), x(i-1))
        return x(5)[1]
    """newton method for v0"""
    def ppp(i):
        return pfinal(i)-1
    from scipy.optimize import newton
    truev0=newton(ppp, 0.5)
    def xstar(i):
        if i==0:
            return np.array((truev0,0))
        return np.dot(A(i),xstar(i-1))
    wa=[]
    for i in range(5):
        wa.append(xstar(i)[0])
    return wa

"""draw plots for w1-w5"""
fig, ax=plt.subplots()
ssfinal=np.linspace(0.2,1,1000)
for i in [1,2,3,4,5]:
    wwa=[]
    for sfinal in ssfinal:
        wwa.append(wages(sfinal)[i-1])
    current_label=r'w{}'.format(i)
    ax.plot(ssfinal, wwa, linewidth=2, alpha=0.6, label=current_label)
ax.legend()
plt.show()
